In [1]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    
from list_prompt_family import ListPromptFamily
from prompt_registry import PROMPT_REGISTRY, ALL_PROMPTS
from wrap_registry import WRAP_REGISTRY
import numpy as np

/Users/johnwu/.pyenv/versions/3.9.1/lib/python3.9/site-packages/torch/onnx/_internal/_beartype.py:36: UserWarning: unhashable type: 'list'
  warnings.warn(f"{e}")


In [2]:
print("Available Prompts:")
for name in PROMPT_REGISTRY["list"]:
    print("-", name)

print("\nAvailable Wrap Styles:")
for name in WRAP_REGISTRY:
    print("-", name)


Available Prompts:
- print
- append
- add_all
- insert_middle
- swap_indices

Available Wrap Styles:
- plain
- list
- interpreter
- system


In [3]:
# Example Prompt 
prompt = PROMPT_REGISTRY["list"]["append"]
case = prompt.create_case(inputs=[[1, 2, 3], 4])

print("Prompt (raw):\n", case.prompt)
print("Ground truth:", case.ground_truth)

wrap_fn = WRAP_REGISTRY["interpreter"]
wrapped_prompt = wrap_fn(prompt.prompt_fn(case.inputs), case.inputs)
print("Wrapped Prompt:\n", wrapped_prompt)

Prompt (raw):
 Append 4 to the end of this list [1, 2, 3]
Ground truth: [1, 2, 3, 4]
Wrapped Prompt:
 Pretend you are a Python interpreter.
TASK: Append 4 to the end of this list [1, 2, 3]
INPUT: [[1, 2, 3], 4]
OUTPUT:


In [4]:
# Generating multiple prompts
prompt.random_input_fn = lambda: [[1, 2, 3], np.random.randint(5, 10)]
cases = prompt.create_cases(n=3)

for c in cases:
    print("\nPrompt:")
    print(c.prompt)
    print("Expected:", c.ground_truth)



Prompt:
Append 8 to the end of this list [1, 2, 3]
Expected: [1, 2, 3, 8]

Prompt:
Append 5 to the end of this list [1, 2, 3]
Expected: [1, 2, 3, 5]

Prompt:
Append 8 to the end of this list [1, 2, 3]
Expected: [1, 2, 3, 8]


In [5]:
family = ListPromptFamily(min_val=1, max_val=5, append_max=15, list_size=4)

cases = family.generate(prompt_name="add_all", wrap_name="list", n=5)

for case in cases:
    print(f"\n{case.task_id}")
    print(case.prompt)
    print("Expected:", case.ground_truth)



add_all-0
Add 2 to every element in this list: [2, 1, 1, 1]
Only output a list, no other information.
List:
Expected: [4, 3, 3, 3]

add_all-1
Add 10 to every element in this list: [1, 3, 4, 1]
Only output a list, no other information.
List:
Expected: [11, 13, 14, 11]

add_all-2
Add 2 to every element in this list: [1, 1, 3, 3]
Only output a list, no other information.
List:
Expected: [3, 3, 5, 5]

add_all-3
Add 9 to every element in this list: [3, 1, 2, 3]
Only output a list, no other information.
List:
Expected: [12, 10, 11, 12]

add_all-4
Add 13 to every element in this list: [3, 2, 1, 4]
Only output a list, no other information.
List:
Expected: [16, 15, 14, 17]


In [6]:
family = ListPromptFamily(min_val=0, max_val=5, append_max=10, list_size=3)

all_cases = family.generate_all(n=3)

for case in all_cases:
    print(f"\n{case.task_id} | wrap: {case.metadata['wrap_name']}")
    print(case.prompt)
    print("Expected:", case.ground_truth)



print-0 | wrap: plain
Print out this list of numbers: [0, 1, 4].
Expected: [0, 1, 4]

print-1 | wrap: plain
Print out this list of numbers: [4, 1, 0].
Expected: [4, 1, 0]

print-2 | wrap: plain
Print out this list of numbers: [4, 0, 2].
Expected: [4, 0, 2]

print-3 | wrap: list
Print out this list of numbers: [4, 3, 2].
Only output a list, no other information.
List:
Expected: [4, 3, 2]

print-4 | wrap: list
Print out this list of numbers: [0, 3, 0].
Only output a list, no other information.
List:
Expected: [0, 3, 0]

print-5 | wrap: list
Print out this list of numbers: [0, 1, 1].
Only output a list, no other information.
List:
Expected: [0, 1, 1]

print-6 | wrap: interpreter
Pretend you are a Python interpreter.
TASK: Print out this list of numbers: [3, 3, 2].
INPUT: [[3, 3, 2]]
OUTPUT:
Expected: [3, 3, 2]

print-7 | wrap: interpreter
Pretend you are a Python interpreter.
TASK: Print out this list of numbers: [2, 0, 0].
INPUT: [[2, 0, 0]]
OUTPUT:
Expected: [2, 0, 0]

print-8 | wrap: 